In [ ]:
---
title: "Weapon target assignment problem"
author: "cbarz"
date: "2023-12-02"
categories: []
draft: true
code-fold: true
toc: true
description: The weapon target assignment problem 
---

## plan

1. gentle introduction
    - including applications of the WTA
1. mathematical formulation
1. pyomo implementation
1. example
1. other solution methods

## Introduction

According to [the wikipedia article](https://en.wikipedia.org/wiki/Weapon_target_assignment_problem) the about the weapon target assignnment problem (WTA) can be formulated as follows:

Given a number of weapons and a number of targets. The weapons are of type $i=1,\ldots ,m$ and $W_i$ denotes the number available weapons of type $i$. Similarly, there are $j=1,\ldots ,n$ targets with value $V_j$. Any of the weapons can be assigned to any target. Each weapon type has a certain probability of destroying each target, given by $p_{ij}$.


## math formulation 

The WTA can be formulated as a non linear integer programm as follows:

$$
\begin{array}{llc}
\min & \sum_j (V_j \prod_i q_{ij}^{x_{ij}}) & \\
s.t. & \sum_j x_{ij} \leq W_i &, \forall i \\
     & x_{ij} \in \mathbb{N}
\end{array}
$$

### objective

minimizing the expected survival value or equivalently maximizing the expeced damange

### (decision) variable

- $x_{ij}$ is the number of weapons of type $i$ assigned to target $j$

### parameter

- $q_{ij}:=(1-p_{ij})$ survival probability of target $j$ for weapon $i$

### constraints

### assumptions


In [ ]:
## NLP example

from pyomo.environ import *

V = 40     # liters
kA = 0.5   # 1/min
kB = 0.1   # l/min
CAf = 2.0  # moles/liter

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.q = Var()

# add a model objective
model.objective = Objective(expr = model.q*V*kA*CAf/(model.q + V*kB)/(model.q + V*kA), sense=maximize)

# compute a solution using ipopt for nonlinear optimization
results = SolverFactory('ipopt').solve(model)
model.pprint()


# print solutions
qmax = model.q()
CBmax = model.objective()
print('\nFlowrate at maximum CB = ', qmax, 'liters per minute.')
print('\nMaximum CB =', CBmax, 'moles per liter.')
print('\nProductivity = ', qmax*CBmax, 'moles per minute.')

1 Var Declarations
    q : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 8.944271909985442 :  None : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40.0*q/(q + 4.0)/(q + 20.0)

2 Declarations: q objective

Flowrate at maximum CB =  8.944271909985442 liters per minute.

Maximum CB = 0.954915028125263 moles per liter.

Productivity =  8.541019662483748 moles per minute.
